### Tokenizing with code

In [1]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4.1-mini")

tokens = encoding.encode("Hi my name is Srini!")
print(tokens)

[12194, 922, 1308, 382, 36013, 2363, 0]


In [2]:
# lets decode 
for token in tokens:
    print(f"{token}: {encoding.decode([token])}")

12194: Hi
922:  my
1308:  name
382:  is
36013:  Sr
2363: ini
0: !


In [11]:
print(encoding.decode([290]))
print(encoding.decode([553]))
print(encoding.decode([3]))

 the
 are
$


### Illusion of "memory"

In [13]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI

In [14]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [15]:
openai = OpenAI()

In [16]:
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant'},
    {'role': 'user', 'content': 'Who won the world series in 2020?'}
]

In [ ]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)

print(response.choices[0].message.content)

The Los Angeles Dodgers won the World Series in 2020. They defeated the Tampa Bay Rays to win the championship.


Ask a follow up question

In [18]:
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant'},
    {'role': 'user', 'content': 'Who won in 2021?'}
]

In [ ]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)

print(response.choices[0].message.content)

Could you please specify which event or competition you are referring to for the year 2021?


We just mentioned about a "world series" in the previous API call, it could not remember it in the next call. Why ??

Because all API calls to an LLM are stateless, that is it does not prior information about the calls we make. Tasks we asked to perform.

As `AI Engineers` it is our responsibility to device a technique to give an impression to the user that LLM has "memory".

In [20]:
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant'},
    {'role': 'user', 'content': 'Who won the world series in 2020?'},
    {'role': 'assistant', 'content': 'The Los Angeles Dodgers won the World Series in 2020.'},
    {'role': 'user', 'content': 'Who won in 2021?'}
]

In [21]:
response =openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)

print(response.choices[0].message.content)

The Atlanta Braves won the World Series in 2021.


### To recap:

- Every call to an LLM is stateless
- We pass in the entire conversation so far in the input prompt, every time
- This gives the illusion that the LLM has memory - it apparently keeps the context of the conversation 
- But this is a trick; it's a by-product of providing the entire conversation, every time an LLM just predicts the most likely next tokens in the sequence; if that sequence contains "My name is Srini" and later "What's my name?" then it will predict.. Srini!
- The ChatGPT product uses exactly this trick - every time you send a message, it's the entire conversation that gets passed in.

"Does that mean we have to pay extra each time for all the conversation so far"

For sure it does. And that's what we WANT. We want the LLM to predict the next tokens in the sequence, looking back on the entire conversation. We want that compute to happen, so we need to pay the electricity bill for it!